In [1]:
from dotenv import load_dotenv
load_dotenv("/home/marshath/play/chainlink/algovate/.env")

import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [20]:
import time
import pickle
import pandas as pd
from IPython.display import Markdown

from langchain.vectorstores import FAISS
from langchain.text_splitter import TokenTextSplitter, RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import FlareChain, RetrievalQAWithSourcesChain
from langchain.chains.router import MultiRetrievalQAChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from tqdm.notebook import tqdm

from algovate.evaluate import run_eval

In [17]:
with open("/home/marshath/play/chainlink/algovate/algovate/data/documents.pkl", "rb") as f:
    documents = pickle.load(f)

In [4]:
def name_desc(
    documents
):
    
    splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)
    documents = splitter.split_documents(documents)

    vectorstore = FAISS.from_documents(documents, embedding=OpenAIEmbeddings())
    retriever = vectorstore.as_retriever()

    template = """
    Given the following extracted parts of a long document and a question, create a name for the document in less than 4 words and description in 20 words.

    QUESTION: {question}
    =========
    {summaries}
    =========
    Name:
    Description:
    """

    PROMPT = PromptTemplate(template=template, 
                            input_variables=["summaries", "question"])
    
    chain = RetrievalQAWithSourcesChain.from_chain_type(ChatOpenAI(temperature=0.), 
                                                        chain_type="stuff", 
                                                        retriever=retriever,
                                                        chain_type_kwargs={"prompt":PROMPT})
    result = chain({
    "question": "What is this document about",
    }, return_only_outputs=True)

    try:
        name = result["answer"].split(":")[1].split("\n")[0].strip()
        desc = result["answer"].split(":")[2].split("\n")[0].strip()
    except:
        name = ""
        desc = ""
    return name, desc, retriever, vectorstore, documents

In [23]:
retriever_infos = []
for doc in tqdm(documents[15:], total=len(documents)):
    name, desc, retriever, vectorstore, d = name_desc([doc])
    retriever_infos.append(
                            {"name":name, 
                            "description":desc, 
                            "retriever":retriever}
                        )
    break

  0%|          | 0/151 [00:00<?, ?it/s]

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": [[2, 29625, 2125, 54878, 35300, 271, 220, 353, 35907, 198, 220, 353, 11443, 315, 1701, 29625, 2125, 54878, 198, 220, 353, 2650, 64815, 21785, 198, 220, 353, 3161, 81949, 10736, 198, 220, 353, 2360, 2494, 10937, 198, 220, 353, 32025, 8335, 198, 220, 353, 8650, 6727, 323, 21523, 426, 55432, 198, 220, 353, 57152, 12, 24375, 6074, 271, 567, 11443, 315, 1701, 29625, 2125, 54878, 271, 19368, 2125, 54878, 1193, 7612, 459, 11572, 11307, 369, 14463, 389, 66286, 627, 2028, 11307, 5764, 279, 7901, 2853, 11, 264, 11668, 15193, 320, 42260, 311, 279, 198, 60599, 3770, 705, 323, 264, 2678, 8521, 6962, 32115, 24043, 369, 6962, 1990, 279, 198, 17969, 323, 279, 19989, 13, 578, 15193, 14573, 988, 279, 54878, 8304, 369, 198, 33325, 287, 323, 16785, 701, 97196, 13, 578, 15193, 35327, 555, 8957, 323, 374, 198, 32340, 389, 1057, 510, 3

In [27]:
Markdown(d[2].page_content)

## Minimum balance

The Chainlink Automation Network is designed to perform your upkeep even when
gas prices spike. The minimum balance in LINK reflects the best estimate of
the cost to perform your upkeep when gas prices spike. To ensure your upkeep
is monitored and performed, ensure that your upkeep's balance is above this
minimum balance.

The minimum balance is calculated using the current fast gas price, the gas
limit you entered for your upkeep, the max gas multiplier, and the for
conversion to LINK. To find the latest value for the `gasCeilingMultiplier`,
see the [Registry Configuration](/chainlink-automation/supported-networks/#configurations) page.

Follow [maintain a minimum balance](/chainlink-automation/manage-upkeeps/#maintain-a-minimum-balance) to ensure that your upkeep is funded.

## Price selection and Gas Bumping

Automation Nodes select the gas price dynamically based on the prices of
transactions within the last several blocks. This optimizes the gas price
based on current network conditions. Automation Nodes are configured to select
a price based on a target percentile.

If the Automation Node does not see the `performUpkeep` transaction get
confirmed within the next few blocks, it automatically replaces the
transaction and bumps the gas price. This process repeats until the
transaction is confirmed.

## ERC-677 Link

### Count number of token